In [1]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import fiona; help(fiona.open)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

Help on function open in module fiona:

open(fp, mode='r', driver=None, schema=None, crs=None, encoding=None, layer=None, vfs=None, enabled_drivers=None, crs_wkt=None, **kwargs)
    Open a collection for read, append, or write
    
    In write mode, a driver name such as "ESRI Shapefile" or "GPX" (see
    OGR docs or ``ogr2ogr --help`` on the command line) and a schema
    mapping such as:
    
      {'geometry': 'Point',
       'properties': [('class', 'int'), ('label', 'str'),
                      ('value', 'float')]}
    
    must be provided. If a particular ordering of properties ("fields"
    in GIS parlance) in the written file is desired, a list of (key,
    value) pairs as above or an ordered dict is required. If no ordering
    is needed, a standard dict will suffice.
    
    A coordinate reference system for collections in write mode can be
    defined by the ``crs`` parameter. It takes Proj4 style mappings like
    
      {'proj': 'longlat', 'ellps': 'WGS84', 'datum': 'W

In [2]:
#Load data to aggregate
df = gpd.read_file("data/properties_distance_final.gpkg", layer = "properties_distance_final")

In [3]:
colonias_gdf = gpd.read_file("data/colonias.geojson", layer='colonias')
colonias_gdf["id"]  = colonias_gdf.index
colonias_gdf.to_crs(6362, inplace = True)

In [4]:
join_df = df.sjoin(colonias_gdf, how="inner")

In [5]:
join_df.groupby("id_right").size().reset_index().sort_values(by=0).reset_index()

,index,id_right,0
0,0,8,1
1,509,1050,1
2,176,356,1
3,508,1049,1
4,503,1042,1
...,...,...,...
696,301,648,228
697,234,480,256
698,175,355,269
699,180,362,274


In [7]:
col_list = join_df.columns[:31].to_list()
col_list.append("id_right")
join_df_1 = join_df[col_list]
join_df_1 = join_df_1.groupby("id_right").mean(numeric_only = True).reset_index()

In [8]:
col_list = join_df.columns[31:].to_list()

join_df_2 = join_df[col_list]
join_df_2 = join_df_2.groupby("id_right").median(numeric_only = True).reset_index()

In [9]:
neighborhood_df = pd.concat([join_df_1,join_df_2.drop("id_right",axis = 1)] ,axis = 1)

#Merge to get geometry of neighborhood
neighborhood_df = pd.merge(neighborhood_df,colonias_gdf, how = "inner", left_on = "id_right", right_on = "id")

neighborhood_df.drop(["index_right","id","id_left","nombre","vcatas_pro_x","vconst_pro_x" ,"vsuelo_pro_x", "ayocon_rem_pro_y","vcatas_pro_y", "vconst_pro_y", "ayocon_rem_pro_x", "vsuelo_pro_y"], axis = 1, inplace = True)
neighborhood_df['price'] = neighborhood_df["price"]/1000000  # Note this is only to more easaly read of results, since R and Python don't automatically use scientific notation

neighborhood_gpd = gpd.GeoDataFrame(neighborhood_df)
neighborhood_gpd = neighborhood_gpd[neighborhood_gpd["lot_size"] < 2500] # We found an a outlier with a very, very large lot size

neighborhood_gpd.to_csv("data/nb_df_non_norm.csv")
neighborhood_gpd.to_file("data/nb_df_non_norm.gpkg", driver = "GPKG", geometry = "geometry")